In [1]:
import torch
import yaml 
import json
import os
import yaml
import random


from model.pl_PairVAE import PlPairVAE
from dataset.datasetPairH5 import PairHDF5Dataset


/home/cdepourt/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create output directory
output_dir = "/projects/pnria/caroline/AUTOFILL/export_vae_pair"
os.makedirs(output_dir, exist_ok=True)

# Params

In [3]:
path_checkpoint = "/projects/pnria/julien/autofill/runs_pair/grid_ag_wlat0.0001_wsaxs2saxs0.04_wsaxs2les0.05_wles2les0.04_wles2saxs0.05/best.ckpt"
path_config = "/projects/pnria/julien/autofill/runs_pair/grid_ag_wlat0.0001_wsaxs2saxs0.04_wsaxs2les0.05_wles2les0.04_wles2saxs0.05/config_model.yaml"

In [4]:


with open(path_config, 'r') as file:
    config = yaml.safe_load(file)



In [5]:
config

{'dataset': {'conversion_dict_path': '/projects/pnria/caroline/AUTOFILL/conversion_dict_all.json',
  'hdf5_file': '/projects/pnria/caroline/AUTOFILL/pair_saxs_les_closest_of_concentration.h5',
  'metadata_filters': {'material': ['ag']},
  'requested_metadata': ['shape', 'material'],
  'sample_frac': 1.0,
  'transform': {'q_les': {'PaddingTransformer': {'pad_size': 500, 'value': 0}},
   'q_saxs': {'PaddingTransformer': {'pad_size': 500, 'value': 0}},
   'y_les': {'MinMaxNormalizer': {},
    'PaddingTransformer': {'pad_size': 500, 'value': 0}},
   'y_saxs': {'MinMaxNormalizer': {},
    'PaddingTransformer': {'pad_size': 500, 'value': 0}}}},
 'devices': '1',
 'experiment_name': 'grid_ag_wlat0.0001_wsaxs2saxs0.04_wsaxs2les0.05_wles2les0.04_wles2saxs0.05',
 'model': {'VAE_LES': {'args': None,
   'path_checkpoint': '/projects/pnria/julien/autofill/runs/grid_ag_les_beta0.0001_etamin1e-07_ld64_bs64/epoch=73-step=1593590.ckpt',
   'path_config': '/projects/pnria/julien/autofill/runs/grid_ag_les

In [6]:
config["dataset"]["requested_metadata"].append("concentration")
config["dataset"]["requested_metadata"].append("dimension1")
config["dataset"]["requested_metadata"].append("dimension2")
config["dataset"]["requested_metadata"].append('opticalPathLength')

In [7]:
config["dataset"]["requested_metadata"]

['shape',
 'material',
 'concentration',
 'dimension1',
 'dimension2',
 'opticalPathLength']

In [8]:
config["dataset"]["hdf5_file"] = "/projects/pnria/caroline/AUTOFILL/pair_saxs_les_closest_of_concentration.h5"

In [9]:
config["dataset"]["conversion_dict_path"] = "/projects/pnria/caroline/AUTOFILL/conversion_dict_all.json"

In [10]:
model = PlPairVAE(config)

INIT VAE SAXS
{'dilation': 1, 'down_channels': [16, 32, 64, 128, 256, 512], 'in_channels': 1, 'input_dim': 500, 'latent_dim': 64, 'output_channels': 1, 'strat': 'y', 'up_channels': [512, 256, 128, 64, 32, 16]}
LOADING CKPT SAXS
INIT VAE LES
{'dilation': 1, 'down_channels': [16, 32, 64, 128, 256], 'in_channels': 1, 'input_dim': 500, 'latent_dim': 64, 'output_channels': 1, 'strat': 'y', 'up_channels': [256, 128, 64, 32, 16]}
LOADING CKPT LES


In [11]:
checkpoint = torch.load(path_checkpoint, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [12]:
dataset = PairHDF5Dataset(
hdf5_file = config["dataset"]["hdf5_file"],
metadata_filters = config["dataset"]["metadata_filters"],
conversion_dict_path = config["dataset"]["conversion_dict_path"],
sample_frac = config["dataset"]["sample_frac"],
transform =  config["dataset"]["transform"],
requested_metadata =  config["dataset"]["requested_metadata"],
)
print("========================================")

/home/cdepourt/PRIVE/AUTOFILL/dataset/datasetPairH5.py:74: UserWarning: Missing requested metadata columns: {'opticalPathLength', 'concentration'}
  warnings.warn(f"Missing requested metadata columns: {missing}")
Applying filters: 100%|██████████| 1/1 [00:00<00:00,  4.28it/s]


╒══════════════════════════════════════════════╕
│ Dataset Initialization Info                 │
╞══════════════════════════════════════════════╡
│ File: /projects/pnria/caroline/AUTOFILL/pair_saxs_les_closest_of_concentration.h5 │
│ Total samples: 41841                      │
│ Samples filtered: 41824                   │
│ Requested fraction: 1.0                    │
│ Fractioned samples: 41824                  │
│ Requested metadata: 4                      │


In [13]:
with open(config["dataset"]["conversion_dict_path"], 'r') as file:
    conversion_dict = json.load(file)

In [14]:
conversion_dict

{'material': {'latex': 0, 'sio2': 1, 'ag': 2},
 'shape': {'sphere': 0, 'cylinder': 1}}

In [15]:
# Sample 1000 random indices from the dataset
random.seed(42)
indices = random.sample(range(len(dataset)), 100)

# Process each selected item
for file_idx, data_idx in enumerate(indices):
    data_item = dataset[data_idx]
    data_metadata = data_item["metadata"]

    org_y_saxs = data_item["data_y_saxs"]
    org_q_saxs = data_item["data_q_saxs"].detach().numpy().flatten()
    org_y_les = data_item["data_y_les"]
    org_q_les = data_item["data_q_les"].detach().numpy().flatten()

    data_item = {k:v.unsqueeze(0) if ("data_" in k and not "min" in k and not "max" in k) else v for k,v in data_item.items()}
    output = model(data_item)

    recon_y_saxs2saxs = output["recon_saxs"].detach().numpy().flatten()
    recon_y_saxs2les = output["recon_saxs2les"].detach().numpy().flatten()
    recon_y_les2saxs = output["recon_les2saxs"].detach().numpy().flatten()
    recon_y_les2les = output["recon_les"].detach().numpy().flatten()

    recon_y_saxs2saxs = recon_y_saxs2saxs * (data_item["data_y_saxs_max"] - data_item["data_y_saxs_min"]) + data_item["data_y_saxs_min"]
    recon_y_les2saxs = recon_y_les2saxs * (data_item["data_y_saxs_max"] - data_item["data_y_saxs_min"]) + data_item["data_y_saxs_min"]

    os.makedirs(os.path.join(output_dir, str(file_idx)), exist_ok=True)
    txt_filename = os.path.join(output_dir, str(file_idx), f"sample_saxs2saxs_idx_{file_idx:04d}.txt")
    with open(txt_filename, "w") as txt_file:
        for org_q_val, recon_val in zip(org_q_saxs, recon_y_saxs2saxs): 
            txt_file.write(f"{org_q_val} {recon_val}\n")

    txt_filename = os.path.join(output_dir, str(file_idx), f"sample_saxs2les_idx_{file_idx:04d}.txt")
    with open(txt_filename, "w") as txt_file:
        for org_q_val, recon_val in zip(org_q_les, recon_y_saxs2les): 
            txt_file.write(f"{org_q_val} {recon_val}\n")

    txt_filename = os.path.join(output_dir, str(file_idx), f"sample_les2saxs_idx_{file_idx:04d}.txt")
    with open(txt_filename, "w") as txt_file:
        for org_q_val, recon_val in zip(org_q_saxs, recon_y_les2saxs): 
            txt_file.write(f"{org_q_val} {recon_val}\n")

    txt_filename = os.path.join(output_dir, str(file_idx), f"sample_les2les_idx_{file_idx:04d}.txt")
    with open(txt_filename, "w") as txt_file:
        for org_q_val, recon_val in zip(org_q_les, recon_y_les2les): 
            txt_file.write(f"{org_q_val} {recon_val}\n")



    # Convert metadata
    converted_metadata = {}
    for k, v in data_metadata.items():
        v = v.cpu().numpy().item()
        if k in conversion_dict:
            inv_conv = conversion_dict[k]
            inv_conv = {v_: k_ for k_, v_ in inv_conv.items()}
            v = inv_conv.get(v, v)
        converted_metadata[k] = v
    yaml_filename = os.path.join(output_dir, str(file_idx), f"sample_{file_idx:04d}.yaml")
    with open(yaml_filename, "w") as yaml_file:
        yaml.dump(converted_metadata, yaml_file)
